# Basics

In [1]:
import threading

In [10]:
# t = threading.Thread()
# t

In [9]:
def task():
    print("hello world, from thread", threading.get_native_id())

t = threading.Thread(target=task)
t.start()
print("hello world main from thread")

hello world, from thread 1253
hello world main from thread


In [17]:
total = 0

def add(count):
    global total
    for i in range(count):
        total += i
    # print("hello world, from thread", threading.get_native_id())

t = threading.Thread(target=add, args=[1_000_000])
t.start()
# bug is a "race condition"
# race between the adding and printing threads
# -- who gets further first determines the result and whether it's correct
print(total)

2955997605


# Day 2

In [1]:
import threading
import torch
import time

In [13]:
# %%time
# unsafe code

total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    for i in range(count):
        total += 1  # critical section

# inc(1000)

start = time.time()
t1 = threading.Thread(target=inc, args=[100_000])
t2 = threading.Thread(target=inc, args=[100_000])
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start, "seconds")
total

0.9196603298187256 seconds


tensor(199875, dtype=torch.int32)

In [5]:
import dis

# dis.dis("")
dis.dis("total += 1")

# load total
# load 1
# add
# store total


  0           0 RESUME                   0

  1           2 LOAD_NAME                0 (total)
              4 LOAD_CONST               0 (1)
              6 BINARY_OP               13 (+=)
             10 STORE_NAME               0 (total)
             12 LOAD_CONST               1 (None)
             14 RETURN_VALUE


In [12]:
# %%time

lock = threading.Lock()
total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    for i in range(count):
        lock.acquire()
        total += 1
        lock.release()
        

# inc(1000)

start = time.time()
t1 = threading.Thread(target=inc, args=[100_000])
t2 = threading.Thread(target=inc, args=[100_000])
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start, "seconds")
total

2.6247355937957764 seconds


tensor(200000, dtype=torch.int32)

# An example

In [19]:
# undafe 
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    lock.acquire()
    success = False
    if bank_accounts[src] >= amount:
        bank_accounts[src] -= amount
        bank_accounts[dst] += amount
        success = True
    print("transferred" if success else "denied")
    lock.release()

In [20]:
transfer("x", "y", 20)

transferred


In [21]:
transfer("x", "z", 10)

denied


In [24]:
# EXCEPTION!
# transfer("w", "x", 50)

In [ ]:
transfer("z", "x", 50) # deadlock because of exception, lock was not released

In [5]:
# safe
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    with lock: # cleaning up resources, lock will be released even if there was an exception
        success = False
        if bank_accounts[src] >= amount:
            bank_accounts[src] -= amount
            bank_accounts[dst] += amount
            success = True
        print("transferred" if success else "denied")

In [3]:
transfer("w", "x", 50) 

KeyError: 'w'

In [4]:
transfer("z", "x", 50) 

transferred
